In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! pip install pytorch torchvision cudatoolkit
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement cudatoolkit (from versions: none)
ERROR: No matching distribution found for cudatoolkit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 1.4 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-2l828xiu
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-2l828xiu
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=c1ea36106fc920c4ce96985286682f86a2fa82b4288c4cd3686ddd28b00dc350
  Stored in directory: /tmp/pip-ephem-wheel-cache-iw2r9jr2/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [10]:
# test code
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/drive/Shareddrives/CS236/final_project/frames/tiger/frame_0.png")).unsqueeze(0).to(device)
text = clip.tokenize(["a tiger", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[9.9609691e-01 2.4044437e-04 3.6626642e-03]]


Text-Video Consistency (CLIP-Score)

In [67]:
import os
import torch
import clip
from PIL import Image
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

#frames_dir = "/content/drive/Shareddrives/CS236/final_project/frames/golden_retriever"
frames_dir = "/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000"
frames = os.listdir(frames_dir)
frames.sort()
frames = frames[0:8]

#embedding = "dog plays in the water"
embedding = "tiger walking in grass"

result = []

for frame in frames:
  print(os.path.join(frames_dir, frame))
  image = preprocess(Image.open(os.path.join(frames_dir, frame))).unsqueeze(0).to(device)
  text = clip.tokenize(embedding).to(device)
  with torch.no_grad():
      image_features = model.encode_image(image)
      text_features = model.encode_text(text)
      cos_similarity = cos(image_features, text_features)
      print(cos_similarity)
      result.append(cos_similarity)
print("result: "+str((sum(result)/len(result)) * 100))


/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_0.png
tensor([0.3113])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_1.png
tensor([0.3002])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_2.png
tensor([0.3077])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_3.png
tensor([0.3011])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_4.png
tensor([0.3064])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_5.png
tensor([0.3095])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_6.png
tensor([0.2956])
/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_walking_30000/frame_7.png
tensor([0.2915])


Text-Video Consistency (CLIP-Temp)

In [70]:
import os
import torch
import clip
from PIL import Image
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

frames_dir = "/content/drive/Shareddrives/CS236/final_project/frames/baseline_no_inv"
frames = os.listdir(frames_dir)
frames.sort()
frames = frames[0:8]

result = []

for i in range(len(frames) - 1):
  image_1 = preprocess(Image.open(os.path.join(frames_dir, frames[i]))).unsqueeze(0).to(device)
  image_2 = preprocess(Image.open(os.path.join(frames_dir, frames[i+1]))).unsqueeze(0).to(device)
  with torch.no_grad():
      image_1_features = model.encode_image(image_1)
      image_2_features = model.encode_image(image_2)
      cos_similarity = cos(image_1_features, image_2_features)
      print(cos_similarity)
      result.append(cos_similarity)
print("result: "+str((sum(result)/len(result)) * 100))

tensor([0.9134])
tensor([0.9701])
tensor([0.9632])
tensor([0.9586])
tensor([0.9617])
tensor([0.9575])
tensor([0.9616])
result: tensor([95.5152])


Warping Error

In [77]:
import cv2
import numpy as np


frames_dir = "/content/drive/Shareddrives/CS236/final_project/main/results_fewshot/frames/tiger_snow_30000"
frames = os.listdir(frames_dir)
frames.sort()
frames = frames[0:8]

result = []

for i in range(len(frames) - 1):

  # Load images (Assuming they are already read into image_1 and image_2)
  image_1 = cv2.imread(os.path.join(frames_dir, frames[i]))
  image_2 = cv2.imread(os.path.join(frames_dir, frames[i+1]))

  # Initialize feature detector (ORB, SIFT, SURF can also be used)
  orb = cv2.ORB_create()

  # Find keypoints and descriptors
  kp1, des1 = orb.detectAndCompute(image_1, None)
  kp2, des2 = orb.detectAndCompute(image_2, None)

  # Create a matcher and find matches (FLANN based or BFMatcher can also be used)
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = matcher.match(des1, des2)

  # Sort matches by distance (best matches first)
  matches = sorted(matches, key=lambda x: x.distance)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros_like(points1)

  for i, match in enumerate(matches):
      points1[i, :] = kp1[match.queryIdx].pt
      points2[i, :] = kp2[match.trainIdx].pt

  # Find homography (transformation matrix)
  H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

  warped_image_1 = cv2.warpPerspective(image_1, H, (image_2.shape[1], image_2.shape[0]))

  error = np.sum((warped_image_1.astype("float") - image_2.astype("float")) ** 2)
  error /= float(warped_image_1.shape[0] * warped_image_1.shape[1])

  result.append(error)

print(str((sum(result)/len(result))))

7756.696088518415
